# Process Synchronization

## Race Condition:
### Scenario: Bus Seat Reservation
A bus has 2 available seats. Two users, Susan and Prajwal, try to reserve seats concurrently:
- Susan wants to book 2 seats.
- Prajwal wants to book 1 seat.

In [2]:
from multiprocessing import Process, Value, current_process
from time import sleep

class Bus:
    def __init__(self, name, available_seats):
        self.name = name
        self.available_seats = available_seats  # No lock, just shared memory

    def reserve(self, need_seats):
        print(f"Process {current_process().name} attempting to reserve {need_seats} seat(s)...")
        print(f"Available seats are: {self.available_seats.value}")
        if self.available_seats.value >= need_seats:
            nm = current_process().name
            sleep(0.5)  # Simulate a longer delay to increase the chance of race condition
            self.available_seats.value -= need_seats  # Update the available seats in shared memory
            print(f"{need_seats} seat(s) are allocated to {nm}")
        else:
            print(f"Sorry! Not enough seats for {current_process().name}")

def main():
    # Create a shared memory object for available seats
    available_seats = Value('i', 2)  # Shared memory, initially 2 seats

    # Create a bus with the shared available seats
    b1 = Bus("Makalu Yatayat", available_seats)

    # Create processes for Susan and Prajwal
    p1 = Process(target=b1.reserve, args=(2,), name="Susan")
    p2 = Process(target=b1.reserve, args=(1,), name="Prajwal")

    # Start processes
    p1.start()
    p2.start()

    # Wait for processes to finish
    p1.join()
    p2.join()

if __name__ == "__main__":
    main()


Process Susan attempting to reserve 2 seat(s)...
Available seats are: 2
Process Prajwal attempting to reserve 1 seat(s)...
Available seats are: 2
2 seat(s) are allocated to Susan
1 seat(s) are allocated to Prajwal


### Issues (Race Condition):
- Simultaneous Access: Both threads may check self.available_seats (2 seats) before it is updated, thinking seats are available.

- Inconsistent Results: Both threads may reserve seats, resulting in negative seat count or overselling.

## 1. Solution with Mutex Lock:
- To solve the race condition and prevent data corruption, we can use a mutex lock to synchronize access to the shared counter variable. This ensures that only one process can modify the counter at a time, preventing the processes from interfering with each other.

### acquire(blocking=True,timeout=-1):
- Blocks until the lock is available.
- Optionally accepts a timeout parameter (default is -1, meaning block indefinitely).
- Returns True when the lock is acquired, False if the timeout expires without acquiring it.

### release():
- Releases the lock, allowing other process to acquire it.
- Must be called by the process holding the lock.
- Raises a RuntimeError if called by a process that doesn't hold the lock.

In [3]:
from multiprocessing import Process, Lock, Value
from time import sleep

class Bus:
    def __init__(self, name, available_seats):
        self.name = name
        self.lock = Lock()  # Create a lock
        self.available_seats = available_seats  # No change here, it's already a shared Value

    def reserve(self, need_seats):
        print(f"Process {current_process().name} attempting to reserve {need_seats} seat(s)...")
        with self.lock:  # Ensure only one process can access this part at a time
            print(f"Available seats are: {self.available_seats.value}")
            if self.available_seats.value >= need_seats:
                nm = current_process().name
                self.available_seats.value -= need_seats  # Update the available seats in shared memory
                print(f"{need_seats} seat(s) are allocated to {nm}")
                sleep(1)  # Simulate processing delay
            else:
                print(f"Sorry! Not enough seats for {current_process().name}")

def main():
    # Create a shared memory object for available seats
    available_seats = Value('i', 2)  # Shared memory, initially 2 seats

    # Create a bus with the shared available seats
    b1 = Bus("Makalu Yatayat", available_seats)

    # Create processes for Susan and Prajwal
    p1 = Process(target=b1.reserve, args=(2,), name="Susan")
    p2 = Process(target=b1.reserve, args=(1,), name="Prajwal")

    # Start processes
    p1.start()
    p2.start()

    # Wait for processes to finish
    p1.join()
    p2.join()

if __name__ == "__main__":
    main()


Process Susan attempting to reserve 2 seat(s)...
Available seats are: 2
2 seat(s) are allocated to SusanProcess Prajwal attempting to reserve 1 seat(s)...

Available seats are: 0
Sorry! Not enough seats for Prajwal


### How the Lock Solves the Issue:
- The with self.lock ensures that only one process executes the reserve method at a time.
- It prevents simultaneous access to the shared available_seats variable, avoiding overselling or inconsistencies.

### Disadvantages of Mutex Lock:
#### i. Cannot be Acquired Multiple Times by the Same Process:

- A mutex lock can only be acquired once by the process that holds it. If the same process tries to acquire it again (in nested calls), it will cause a deadlock.
- This makes it unsuitable for recursive or reentrant operations that require the process to enter the locked section multiple times.


### Why we use multiple locks on same resource?
- Multiple locks on the same resource can happen due to mistakes by the developer, especially if they are unaware of how other parts of the code are locking the same resource. This can lead to issues like deadlocks or performance inefficiencies.

## 2. Solution with Rlock:

### 2.1 Problem of Mutex Lock in Processes:

In [4]:
from multiprocessing import Process, Lock, Value
from time import sleep

# Shared resource
counter = Value('i', 0)  # Create a shared integer

# Create a mutex lock
mutex = Lock()

def increment():
    global counter
    mutex.acquire()  # Acquire lock for the first time
    counter.value += 1
    print(f"Counter after first increment: {counter.value}")

    sleep(1)  # Simulate some work

    mutex.acquire()  # Attempt to acquire the lock again (This will cause a deadlock)
    counter.value += 1
    print(f"Counter after second increment: {counter.value}")

    mutex.release()  # Release lock
    mutex.release()  # Release lock again

if __name__ == "__main__":
    # Start a process
    process = Process(target=increment)
    process.start()
    process.join()

    print(f"Final Counter Value: {counter.value}")


Counter after first increment: 1


Process Process-7:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-4e7cc7a85bce>", line 18, in increment
    mutex.acquire()  # Attempt to acquire the lock again (This will cause a deadlock)
KeyboardInterrupt


KeyboardInterrupt: 

### Issue:
- When the increment() function runs, it acquires the lock and increments the counter.
- After the first counter += 1, the function attempts to acquire the same lock again with mutex.acquire().
- The Mutex lock does not allow the same process to acquire the lock again, and the second mutex.acquire() causes a deadlock.
- The process is waiting for itself to release the lock, and the program hangs indefinitely.

### How RLock Solves the Issue:
- RLock (Reentrant Lock) allows the same process to acquire the same lock multiple times without causing a deadlock.
- It works by tracking the number of times a process has acquired the lock.
- The process can call acquire() multiple times, and it must call release() the same number of times to release the lock.

### 2.1 Solution Using RLock:

In [5]:
from multiprocessing import Process, RLock, Value
from time import sleep

# Shared resource
counter = Value('i', 0)  # Create a shared integer

# Create a reentrant lock (RLock)
rlock = RLock()

def increment():
    with rlock:  # Acquire the lock for the first time
        counter.value += 1
        print(f"Counter after increment: {counter.value}")

        with rlock:  # Successfully acquire the lock again (No deadlock)
            counter.value += 1
            print(f"Counter after second increment: {counter.value}")

if __name__ == "__main__":
    # Start a process
    process = Process(target=increment)
    process.start()
    process.join()

    print(f"Final Counter Value: {counter.value}")


Counter after increment: 1
Counter after second increment: 2
Final Counter Value: 2


### Key Points for Processes:
- Matching acquire and release: For each acquire(), there must be a corresponding release().
- Cannot release more times than acquired: Calling release() more times than acquire() raises a RuntimeError.
- Cannot release without acquiring: You cannot release() a lock without first calling acquire().
- Multiple acquires by the same process: A process can acquire the lock multiple times (using RLock), but it must release it the same number of times.
- Process ownership: Only the process that acquired the lock can release it; other processes cannot release it.

### Limitation of Locks and RLocks for Processes:
- Single Process Access: Locks (including RLocks) only allow one process to acquire the lock at any given time. This means only one process can execute the critical section of code while others are blocked, limiting concurrency.

## 3. Solution with Semaphore for Processes:
- A semaphore is a synchronization primitive used to control access to a shared resource by multiple processes in a concurrent system. It maintains a counter, which indicates the number of available resources or the number of processes that can access the critical section.
### 3.1 Unbounded Semaphore (No Limit):
#### What is it?
- An unbounded semaphore doesn’t have a fixed limit. It starts with an initial number of tickets, but you can keep releasing tickets as long as you want.
- Example: Imagine you have 3 tickets to start with, but after a process finishes, you can release as many tickets as you like, and new processes can keep acquiring them.
#### How it works:
- If you initialize the semaphore with 3 permits, processes will be able to acquire permits like before.
- But if you release more tickets than you started with, the semaphore counter can grow. It doesn't throw an error for releasing more tickets than initially acquired.

In [ ]:
from multiprocessing import Process, Semaphore, current_process
import time

# Unbounded semaphore with 3 permits
semaphore = Semaphore(3)

def use_resource():
    # Correct way to get the process name
    current_process_name = current_process().name
    print(f"{current_process_name} is trying to acquire a ticket...")
    semaphore.acquire()  # Try to acquire a permit (ticket)
    print(f"{current_process_name} is using the resource.")
    time.sleep(1)  # Simulate using the resource
    print(f"{current_process_name} is releasing the ticket.")
    semaphore.release()  # Release the permit (ticket)

def main():
    # Create 5 processes (more than the permits)
    processes = [Process(target=use_resource, name=f"Process-{i}") for i in range(5)]

    # Start the processes
    for p in processes:
        p.start()

    # Wait for all processes to finish
    for p in processes:
        p.join()

    print("All processes have finished.")

if __name__ == "__main__":
    main()


Process-0 is trying to acquire a ticket...
Process-0 is using the resource.
Process-1 is trying to acquire a ticket...
Process-1 is using the resource.
Process-2 is trying to acquire a ticket...
Process-2 is using the resource.

Process-3 is trying to acquire a ticket...Process-4 is trying to acquire a ticket...
Process-0 is releasing the ticket.
Process-3 is using the resource.
Process-1 is releasing the ticket.
Process-4 is using the resource.
Process-2 is releasing the ticket.
Process-3 is releasing the ticket.
Process-4 is releasing the ticket.
All processes have finished.


### Explanation:
- Semaphore starts with 3 tickets.
- 3 processes acquire tickets, so the counter goes from 3 → 0.
- After using the resource, each process releases a ticket.
- After releasing 3 tickets, the semaphore counter becomes 3 again.
- If 4 tickets are released (e.g., releasing a 4th ticket when the counter is 0), the semaphore counter goes to 4.

### 3.2 Bounded Semaphore (Fixed Limit) for Processes:
#### What is it?
- A bounded semaphore is a semaphore that has a fixed maximum limit on how many processes can access the resource at the same time.
- Example: Imagine you have 3 tickets (permits) for 3 people to access a printer. Once 3 people are using the printer, the 4th person has to wait because all tickets are taken.
#### How it works:
- If you initialize a bounded semaphore with 3 permits, only 3 processes can access the resource at once.
- Once the limit is reached, new processes will have to wait until someone finishes and releases a ticket.

In [ ]:
from multiprocessing import Process, BoundedSemaphore, current_process
import time

# Bounded semaphore with 3 permits
semaphore = BoundedSemaphore(3)

def use_resource():
    # Correct way to get the process name
    current_process_name = current_process().name
    print(f"{current_process_name} is trying to acquire a ticket...")
    semaphore.acquire()  # Try to acquire a permit (ticket)
    print(f"{current_process_name} is using the resource.")
    time.sleep(1)  # Simulate using the resource
    print(f"{current_process_name} is releasing the ticket.")
    semaphore.release()  # Release the permit (ticket)

def main():
    # Create 5 processes (more than the permits)
    processes = [Process(target=use_resource, name=f"Process-{i}") for i in range(5)]

    # Start the processes
    for p in processes:
        p.start()

    # Wait for all processes to finish
    for p in processes:
        p.join()

    print("All processes have finished.")

if __name__ == "__main__":
    main()


Process-0 is trying to acquire a ticket...
Process-0 is using the resource.
Process-1 is trying to acquire a ticket...

Process-1 is using the resource.Process-2 is trying to acquire a ticket...
Process-2 is using the resource.
Process-3 is trying to acquire a ticket...
Process-4 is trying to acquire a ticket...
Process-0 is releasing the ticket.
Process-3 is using the resource.
Process-1 is releasing the ticket.
Process-4 is using the resource.
Process-2 is releasing the ticket.
Process-3 is releasing the ticket.
Process-4 is releasing the ticket.
All processes have finished.


### Explanation:
- Semaphore starts with a fixed number of tickets (e.g., 3).
- Processes acquire tickets: The counter decreases as processes acquire tickets. For example, 3 processes acquire tickets, so the counter goes from 3 → 0.
- Once the semaphore reaches 0, no more processes can acquire a ticket. They must wait until a ticket is released.
- After a process finishes using the resource, it releases a ticket. The counter increases by 1 for each release.
- If a process tries to release more tickets than the initial limit (3), the semaphore will not allow it, and the ticket count will remain within the initial limit.

## How semaphore eliminate race condition for process?
### What Semaphore Can Do:
- Control access: Semaphore allows multiple processes to access a shared resource concurrently, but it limits the number of processes that can access the resource at any given time.
- Concurrency management: By limiting the number of processes, it can prevent the system from being overwhelmed by too many processes trying to access a resource simultaneously.
### What Semaphore Cannot Do:
- Prevent race conditions: Semaphore by itself does not guarantee that multiple processes won't modify shared data concurrently, leading to race conditions. Multiple processes can still access and modify shared data simultaneously unless mutual exclusion (via locks) is implemented.

### Eliminating Race Conditions with Semaphore + Lock
To eliminate race conditions while using semaphores, semaphores control the concurrency (how many processes can access a resource), and locks (like Lock) ensure mutual exclusion when processes modify the shared resource.

- Semaphore: Controls how many processes can access the critical section at a time.
- Lock: Ensures that only one process can modify the shared resource at a time, thus preventing race conditions.

In [ ]:
from multiprocessing import Process, Semaphore, Lock, Value, current_process
import time

# Shared resource (counter)
counter = Value('i', 0)

# Semaphore allowing up to 3 processes to access the shared resource
semaphore = Semaphore(3)

# Lock to ensure mutual exclusion when modifying the shared resource
lock = Lock()

def increment():
    semaphore.acquire()  # Limit the number of processes accessing the resource

    with lock:  # Ensure only one process modifies the counter at a time
        current_value = counter.value
        time.sleep(0.1)  # Simulate some processing
        counter.value = current_value + 1
        print(f"Counter incremented by {current_process().name}. New counter: {counter.value}")

    semaphore.release()  # Allow another process to access the resource

def main():
    # Create and start multiple processes
    processes = []
    for i in range(5):
        process = Process(target=increment, name=f"Process-{i+1}")
        processes.append(process)
        process.start()

    # Wait for all processes to finish
    for process in processes:
        process.join()

    print(f"Final Counter Value: {counter.value}")

if __name__ == "__main__":
    main()


Counter incremented by Process-1. New counter: 1
Counter incremented by Process-2. New counter: 2
Counter incremented by Process-3. New counter: 3
Counter incremented by Process-4. New counter: 4
Counter incremented by Process-5. New counter: 5
Final Counter Value: 5
